In [1]:
import ltlcross_wrapper

tmp_name = "$LCW_TMP.in"
tmp_out = "$LCW_TMP.out"

goal_bin = "$LCW_GOAL_BIN"

tgba = "ltl2tgba %f | "
sba  = "ltl2tgba %f | autfilt -B | "

save_to_file = f"cat > {tmp_name}"

autfilt_no  = f" && cat {tmp_out} > %O"
autfilt_yes = f" && autfilt --small --tgba {tmp_out} > %O"
cleanup = f" && rm {tmp_name} {tmp_out}"

def goal_cmd(algo, options=""):
    return f"{goal_bin} batch '$temp = complement -m {algo} {options} `echo {tmp_name}`; save -c HOAF $temp `echo {tmp_out}`;'"

def goal_pipeline(algo, options=""):
    return f"{sba} {save_to_file} && {goal_cmd(algo, options)}"

fribourg = goal_pipeline("fribourg")
piterman = goal_pipeline("piterman")

simpl = " | autfilt --small --tgba "

tools = {
    "fribourg.yes"    : fribourg + autfilt_yes + cleanup,
    "piterman.yes"    : piterman + autfilt_yes + cleanup,
    "ltl2tgba" : "ltlfilt --negate -f %f | ltl2tgba > %O",
}

In [4]:
m = ltlcross_wrapper.GoalModulizer(goal_root="GOAL-20200506", tools=tools, formula_file="literature_nd.ltl", name="test.goal", chunk_size=1)

In [5]:
m.run()

In [11]:
a = ltlcross_wrapper.ResAnalyzer("test.goal.csv")
a.get_error_counts()

,timeout,parse error,incorrect,crash,no output
fribourg.yes,1,0,0,0,0
piterman.yes,1,0,0,0,0
